In [1]:
beyts = []

with open('ferdousi.txt', 'r', encoding='utf-8') as f:
    next(f)
    next(f)
    for line in f:
        beyts.append(line.strip())
pairs = []
for i in range(len(beyts) // 2):
    pairs.append((beyts[2 * i], beyts[2 * i + 1]))
pairs

[('به نام خداوند جان و خرد', 'کزین برتر اندیشه برنگذرد'),
 ('خداوند نام و خداوند جای', 'خداوند روزی ده رهنمای'),
 ('خداوند کیوان و گردان سپهر', 'فروزنده ماه و ناهید و مهر'),
 ('ز نام و نشان و گمان برترست', 'نگارندهٔ بر شده پیکرست'),
 ('به بینندگان آفریننده را', 'نبینی مرنجان دو بیننده را'),
 ('نیابد بدو نیز اندیشه راه', 'که او برتر از نام و از جایگاه'),
 ('سخن هر چه زین گوهران بگذرد', 'نیابد بدو راه جان و خرد'),
 ('خرد گر سخن برگزیند همی', 'همان را گزیند که بیند همی'),
 ('ستودن نداند کس او را چو هست', 'میان بندگی را ببایدت بست'),
 ('خرد را و جان را همی سنجد اوی', 'در اندیشهٔ سخته کی گنجد اوی'),
 ('بدین آلت رای و جان و زبان', 'ستود آفریننده را کی توان'),
 ('به هستیش باید که خستو شوی', 'ز گفتار بی کار یکسو شوی'),
 ('پرستنده باشی و جوینده راه', 'به ژرفی به فرمانش کردن نگاه'),
 ('توانا بود هر که دانا بود', 'ز دانش دل پیر برنا بود'),
 ('از این پرده برتر سخن گاه نیست', 'ز هستی مر اندیشه را راه نیست'),
 ('کنون ای خردمند وصف خرد', 'بدین جایگه گفتن اندرخورد'),
 ('کنون تا چه داری بیار از خرد', '

In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/gpt2-fa")
tokenizer.add_special_tokens({
    'pad_token': '<pad>',
    'sep_token': '<sep>',
})
model = AutoModelForCausalLM.from_pretrained("HooshvareLab/gpt2-fa").to(device)

In [4]:
sum(p.numel() for p in model.parameters())

118099200

In [5]:
pairs[0][0]

'به نام خداوند جان و خرد'

In [6]:
def generate(text):
    encoded = tokenizer(text, return_tensors='pt')
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=30, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate(pairs[0][0]))

به نام خداوند جان و خرد است. این کتاب به زبان‌های انگلیسی، فرانسوی، آلمانی، ایتالیایی، اسپانیایی، ترکی استانبولی، روسی، عربی


In [7]:
from torch.utils.data import Dataset

class BeytDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
        self.input_ids = []
        self.attention_masks = []
        for pair in pairs:
            text = pair[0] + '<sep>' + pair[1]
            encoded = tokenizer(text, add_special_tokens=True, max_length=30, return_tensors='pt', padding='max_length', truncation=True)
            self.input_ids.append(encoded['input_ids'][0])
            self.attention_masks.append(encoded['attention_mask'][0])

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx]

In [8]:
from torch.utils.data import DataLoader, random_split

dataset = BeytDataset(pairs)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [9]:
print(dataset[0][0])
print(tokenizer.decode(dataset[0][0], skip_special_tokens=False))

tensor([  490,   561,  6733,  1305,   293,  2964,     9,  2639,   297,  3206,
         6188, 29631,   383,   343,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1])
به نام خداوند جان و خرد<sep>کزین برتر اندیشه برنگذرد<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [10]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

optimizer = optimizer = AdamW(model.parameters(), lr=5e-4, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=20, num_training_steps=len(train_loader) * 3)

In [11]:
from tqdm import tqdm

model.train()
for epoch in range(3):
    pbar = tqdm(train_loader)
    for input_ids, attention_mask in pbar:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        model.zero_grad()
        optimizer.zero_grad()
        output = model(input_ids, labels=input_ids, attention_mask=attention_mask, token_type_ids=None)
        loss = output.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        pbar.set_description(f'Loss: {loss.item():.4f}')
    print(generate(pairs[0][0]))

torch.save(model.state_dict(), 'beyt_model.pth')

Loss: 1.8638: 100%|██████████| 1241/1241 [04:29<00:00,  4.60it/s]


به نام خداوند جان و خردخردمند و بیدار و تن بشردز کژی و خورد نشمرد و ننگردد خرد


Loss: 1.2768: 100%|██████████| 1241/1241 [04:29<00:00,  4.61it/s]


به نام خداوند جان و خردخداوند شمشیر و کوپال و خودز داد و فزاید نشاید به گردون کشان


Loss: 1.4229: 100%|██████████| 1241/1241 [04:30<00:00,  4.59it/s]


به نام خداوند جان و خردخردمند و بیدار و گرد و بردهشیوار و بسیاردان گرد بردش به گرد


In [14]:
from nltk.translate.bleu_score import corpus_bleu
import numpy as np

model.eval()
with torch.no_grad():
    pbar = tqdm(test_loader)
    bleu_scores = []
    perplexities = []
    for input_ids, attention_mask in pbar:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        output = model(input_ids, labels=input_ids, attention_mask=attention_mask, token_type_ids=None)
        loss = output.loss
        perplexity = np.exp(loss.item())
        perplexities.append(perplexity)
        output = model.generate(input_ids, attention_mask=attention_mask, max_length=50, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, pad_token_id=tokenizer.eos_token_id)
        generated = tokenizer.batch_decode(output, skip_special_tokens=True)
        references = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        bleu_score = corpus_bleu([[ref] for ref in references], generated)
        bleu_scores.append(bleu_score)
        pbar.set_description(f'BLEU: {bleu_score:.4f}, Perplexity: {perplexity:.4f}')
    print(f'BLEU: {sum(bleu_scores) / len(bleu_scores):.4f}, Perplexity: {sum(perplexities) / len(perplexities):.4f}')

BLEU: 0.4725, Perplexity: 4.1981: 100%|██████████| 311/311 [41:37<00:00,  8.03s/it]

BLEU: 0.4897, Perplexity: 5.3859


In [20]:
import pandas as pd
random_beyts = np.random.choice(len(test_dataset), 10)
first_mesras = []
second_mesras = []
generated_second_mesras = []
for idx in random_beyts:
    input_ids = test_dataset[idx][0].unsqueeze(0).to(device)
    attention_mask = test_dataset[idx][1].unsqueeze(0).to(device)
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=50, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, pad_token_id=tokenizer.eos_token_id)
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    first_mesra, second_mesra = tokenizer.decode(input_ids[0], skip_special_tokens=False).split('<sep>')
    first_mesras.append(first_mesra)
    second_mesras.append(second_mesra.split('<pad>')[0])
    generated_second_mesras.append(generated[len(first_mesra) + 1:])
df = pd.DataFrame({
    'first_mesra': first_mesras,
    'second_mesra': second_mesras,
    'generated_second_mesra': generated_second_mesras,
})
df

,first_mesra,second_mesra,generated_second_mesra
0,سراسر زمانه پر از جنگ بود,به جویندگان بر جهان تنگ بود,ه جویندگان بر جهان تنگ بودز بخت جهاندار دور از...
1,عنان کرد پیچان براه گریز,برآمد ز گودرزیان رستخیز,رآمد ز گودرزیان رستخیزپر از درد وپر آژنگ وسترگ...
2,که پیغمبر آمد بلهراسپ داد,پذیرفت زان پس بگشتاسپ داد,ذیرفت زان پس بگشتاسپ دادجهاندار داد و پیروز و ...
3,فرستاده بشنید و آمد چو باد,به قیصر بر آن گفتها کرد یاد,ه قیصر بر آن گفتها کرد یادز شادجهاندار پیروز و...
4,چه سازی درنگ اندرین جای تنگ,که شد تنگ بر تو سرای درنگ,ه شد تنگ بر تو سرای درنگنگیرد درنگ از درنگ و ب...
5,که با آهوان گفت غرم ژیان,که گر دشت گردد همه پرنیان,ه گر دشت گردد همه پرنیان�برآرد جهان از کهان و ...
6,ببیژن سپردی و بگریستی,بران شوربختی همی زیستی,ران شوربختی همی زیستیهمی سوختییکه ننگی همی بشو...
7,دهن گر بماند ز خوردن تهی,ازان به که ناساز خوانی نهی,زان به که ناساز خوانی نهینشهیبهی و به مردی و گ...
8,پزشکان داننده را خواندند,به نزدیک ناهید بنشاندند,ه نزدیک ناهید بنشاندندورا خواندندشاندندش خواند...
9,سپهدار گودرز کشواد رفت,به نزدیک خسرو خرامید تفت,ه نزدیک خسرو خرامید تفتبگفت آنچ بشنید و نامه ب...
